In [2]:
!python -V

Python 3.7.3


# Google スニペットを入手しよう

参考： https://qiita.com/zak_y/items/42ca0f1ea14f7046108c#comments

In [1]:
# 関数定義
import os
import datetime
import json

from time import sleep
from googleapiclient.discovery import build

GOOGLE_API_KEY          = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
CUSTOM_SEARCH_ENGINE_ID = "000000000000000000000:xxxxxxxxxxx"

DATA_DIR = 'data'

def makeDir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

def get_data(qid, query, answer, dnum=10):# dnum:データ数.1page目の上から取ってくる数を指定． この引数を持つ関数にしたい．
    today = datetime.datetime.today().strftime("%Y%m%d")
    timestamp = datetime.datetime.today().strftime("%Y/%m/%d %H:%M:%S")

    makeDir(DATA_DIR)# 関数の中から外の変数は見れる

    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY) # このインスタンスが， どんなことできるのか知りたい

    page_limit = 1# どうやら，一度に10件までしかデータを取得できないみたいなので，forを回す必要がある．
    start_index = 1# データを取得し始めるindex. 1なので，頭から．
    response = []
    for n_page in range(0, page_limit):
        try:# 例外を捉える部分．Errorが無ければそのまま実行．あれば exceptへ．
            sleep(1)# 1秒待つ．何故必要かは不明．
            response.append(service.cse().list(# responseにデータを格納.
                q=query,
                cx=CUSTOM_SEARCH_ENGINE_ID,
                lr='lang_ja',
                num=dnum,# データ数. [1,10].
                start=start_index
            ).execute())
            start_index = response[n_page].get("queries").get("nextPage")[0].get("startIndex")
        except Exception as e:
            print(e)
            break
    
    # レスポンスをjson形式で保存
    save_response_dir = os.path.join(DATA_DIR, 'response')
    makeDir(save_response_dir)
    out = {'snapshot_ymd': today, 'snapshot_timestamp': timestamp, 'answer': answer, 'response': []}
    out['response'] = response
    jsonstr = json.dumps(out, ensure_ascii=False)
    with open(os.path.join(save_response_dir, 'qid_' + str(qid) + '.json'), mode='w') as response_file:
        response_file.write(jsonstr)
        
def make_data(qid):
    #today = datetime.datetime.today().strftime("%Y%m%d")

    response_filename = os.path.join(DATA_DIR, 'response', 'qid_' + str(qid) + '.json')
    with open(response_filename, 'r') as response_file:
        response_json = response_file.read()
        response_tmp = json.loads(response_json)
        ymd = response_tmp['snapshot_ymd']
        response = response_tmp['response']
    
    results = []
    cnt = 0
    for one_res in range(len(response)):# len(*)は,qidにつき1.あんまり意味ない．qidにつき11件以上データ取ったらそうでもない？
        if 'items' in response[one_res] and len(response[one_res]['items']) > 0:
            #print(response[one_res]['queries']['request'][0]['searchTerms'])
            answer = response_tmp['answer']# 僕の書き方依存
            question  = response[one_res]['queries']['request'][0]['searchTerms']
            results.append({'qid': qid, 'ymd': ymd, 'question': question, 'answer': answer, 'documents': []})#'title': title, 'link': link, 'snippet': snippet}) 
            for i in range(len(response[one_res]['items'])):
                cnt += 1
                display_link = response[one_res]['items'][i]['displayLink']
                title        = response[one_res]['items'][i]['title']
                link         = response[one_res]['items'][i]['link']
                snippet      = response[one_res]['items'][i]['snippet'].replace('\n', '')
                #print(results[one_res]['documents'])
                results[one_res]['documents'].append({'no': cnt, 'display_link': display_link, 'link': link, 'title': title, 'text': snippet})
                #results.append({'no': cnt, 'ymd': ymd, 'display_link': display_link, 'link': link, 'documents': [{'title': tilte, 'text': snippet}]})#'title': title, 'link': link, 'snippet': snippet}) 
    save_results_dir = os.path.join(DATA_DIR, 'results')
    makeDir(save_results_dir)
    
    # レスポンスをjson形式で保存
    jsonstr = json.dumps(results, ensure_ascii=False)
    with open(os.path.join(save_results_dir, 'qid_' + str(qid) + '.json'), mode='w') as results_file:
        results_file.write(jsonstr)

## (q,a)取得

In [2]:
import gzip
import json

dataset = []
with gzip.open("dataset/all-v1.0.json.gz", "rt", encoding="utf-8") as fp:
    for line in fp:
        data = json.loads(line)
        if data["documents"]:
            dataset.append(data)

## Googleスニペットを入手
3月-11月は16時がリセットタイミングらしい． 1query 10 件データ取ってくるなら， 100query/day. 

In [3]:
start = 400
dataset[start]['qid']

428

In [4]:
hoge# うっかり実行しないようにhoge
for i in range(start, start + 100):
    qid, q, a = dataset[i]['qid'], dataset[i]['question'], dataset[i]['answer']
    get_data(qid, q, a)

'NoneType' object is not subscriptable


In [39]:
# 取得したデータを確認
def view_data(qid):
    dataset = []
    save_response_dir = os.path.join(DATA_DIR, 'response')
    with open(os.path.join(save_response_dir, 'qid_' + str(qid) + '.json'), mode='r') as fp:
        for line in fp:
            data = json.loads(line)
            dataset.append(data)
        return dataset

## 加工整形処理

qid が連続じゃないので，元データのファイルの読み込みに，少し工夫が必要になった

In [3]:
start = 10
dnum = 90
dataset[start]['qid']

12

In [4]:
hoge #うっかり実行しないようにhgoe
for i in range(start, start + dnum):
    make_data(dataset[i]['qid'])

## 取得したデータを確認

In [5]:
#DATA_DIR = 'data'
save_response_dir = os.path.join(DATA_DIR, 'results')
results = []
qid = 1
with open(os.path.join(save_response_dir, 'qid_' + str(qid) + '.json'), mode='r') as fp:
    for line in fp:
        result = json.loads(line)
        results.append(result)

データを見てみて
- snippet の末には \xa0... がある．
- 自分で Google 検索を行った結果と少し異なる．サイトの表示順が違った．APIの検索方法とかは調べた方が良さそう．
- スニペットの選択方法が気になる．
- クエリに対して，何件スニペットを取ってくるかは未定(今回なら10件だけ取ってきた．取得したコードのページ1とした部分.1ページに表示されるサイト数は10件)

In [55]:
results[0][0].keys() # results[1][2]. 1:qid, 2:qidに対応するデータ.10件以上とると，index:0以外も使う．

dict_keys(['qid', 'ymd', 'question', 'answer', 'documents'])

In [6]:
a =results[0][0]['documents']
len(a)

10

In [9]:
a[0] 

{'no': 1,
 'display_link': 'www.neomag.jp',
 'link': 'https://www.neomag.jp/mag_navi/column/column005.html',
 'title': '磁石ナビ | 磁石につく金属の種類と関係性について | 磁石のお話',
 'text': '磁石につく最も身近な金属は鉄です。鉄のほかには、ニッケルやコバルトなどが挙げられます。ステンレスにも磁石がつきますが、これは中に含まれ\xa0...'}

In [51]:
# データ確認
results[0]

[{'qid': 1,
  'ymd': '20200823',
  'question': '鉄にくっつく物体',
  'answer': '磁石',
  'documents': [{'no': 1,
    'display_link': 'www.neomag.jp',
    'link': 'https://www.neomag.jp/mag_navi/column/column005.html',
    'title': '磁石ナビ | 磁石につく金属の種類と関係性について | 磁石のお話',
    'text': '磁石につく最も身近な金属は鉄です。鉄のほかには、ニッケルやコバルトなどが挙げられます。ステンレスにも磁石がつきますが、これは中に含まれ\xa0...'},
   {'no': 2,
    'display_link': 'www.try-it.jp',
    'link': 'https://www.try-it.jp/chapters-1521/sections-1522/lessons-1523/practice-4/',
    'title': '【中1理科】「金属の3つの性質」(練習編) | 映像授業のTry IT ...',
    'text': '磁石にくっつくのは、金属のうち鉄だけ. 中１ 理科化学1 練習(2) 答えなし. lecturer_avatar. (2)は、それぞれの 物体 が電気を通すかどうかと、磁石につくかどうかを答える問題です。 金属に共通する性質として、 伝導性 というものがありま\xa0...'},
   {'no': 3,
    'display_link': 'www.neomag.jp',
    'link': 'https://www.neomag.jp/mag_navi/column/column018.html',
    'title': '磁石ナビ | 磁石の磁力を遮断する方法とは | 磁石のお話',
    'text': '例えば、磁石と鉄をくっつけると、鉄も磁石の力を帯びて釘などを引きつけるようになります。しかし、磁石を鉄から引き剥がせば、鉄は元に戻り、釘を近づけてもくっつくことはありません。これが軟磁性\xa0...'},
   {'no': 4,
    'di